### (Optional) Install Watson libraries

In [ ]:
!pip install --upgrade "ibm-watson>=4.1.0"

### Speech credentials

In [ ]:
credentials = {
  "apikey": "{API-Key}",
  "url": "{URL}" #usually "https://stream.watsonplatform.net/speech-to-text/api"
}

### Setup and configure speech credentials

In [ ]:
from ibm_watson import SpeechToTextV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator(credentials['apikey'])
speech_to_text = SpeechToTextV1(
    authenticator=authenticator
)

speech_to_text.set_service_url(credentials['url'])


### (Optional) List all the available speech models

In [ ]:
import json

speech_models = speech_to_text.list_models().get_result()
print(json.dumps(speech_models, indent=2))

### (Optional) Print out the details of the Broadband model

In [ ]:
import json
#speech_models = speech_to_text.list_models().get_result()
#print(json.dumps(speech_models, indent=2))
speech_model = speech_to_text.get_model('en-US_BroadbandModel').get_result()
print(json.dumps(speech_model, indent=2))

### Transcribe using the base speech (Broadband US English) model

In [ ]:
from os.path import join, dirname

files = ['WebEx Video Data/wmv files/04-09-19.flac']
for file in files:
    with open(join(dirname('.'), './.', file),
                   'rb') as audio_file:
        speech_recognition_results = speech_to_text.recognize(
            audio=audio_file,
            content_type='audio/flac',
            timestamps=True,
            word_alternatives_threshold=0.9
        ).get_result()
    #print(json.dumps(speech_recognition_results, indent=2))

### (Optional) Print the transcription

In [ ]:
transcript = ''
for chunks in speech_recognition_results['results']:
    if 'alternatives' in chunks.keys():
        alternatives = chunks['alternatives'][0]
        if 'transcript' in alternatives:
            transcript = transcript + alternatives['transcript']
print(transcript)

In [ ]:
speech_recognition_results_US_BroadbandModel = speech_recognition_results

### (Optional) Print the customization ID

In [ ]:
# Custom Language model
language_model = speech_to_text.create_language_model(
    'TSL language model',
    'en-US_BroadbandModel',
    description='TSL Production Line QA').get_result()
print(json.dumps(language_model, indent=2))

#speech_to_text.delete_language_model(language_model['customization_id'])

### (Optional) Print all the language models

In [ ]:
language_models = speech_to_text.list_language_models().get_result()
print(json.dumps(language_models, indent=2))

In [ ]:
custom_lang_broadband_model_id = '{Custom-Language-Model-ID}'

### Upgrade the language model

In [ ]:
speech_to_text.upgrade_language_model(custom_lang_broadband_model_id)

### Add text corpus file for language training

In [ ]:
with open(join(dirname('.'), './.', '{filename.txt}'),
               'rb') as corpus_file:
    speech_to_text.add_corpus(
        custom_lang_broadband_model_id,
        '{model-name}',
        corpus_file
    )

### (Optional) Print the details of the corpora

In [ ]:
corpora = speech_to_text.list_corpora(custom_lang_broadband_model_id).get_result()
print(json.dumps(corpora, indent=2))

### Print all the custom words

In [ ]:
words = speech_to_text.list_words(custom_lang_broadband_model_id).get_result()
print(json.dumps(words, indent=2))

### Add grammar words

In [ ]:
with open(join(dirname('.'), './.', '{localtion-path}/productionWords.abnf'),
               'rb') as grammar_file:
    speech_to_text.add_grammar(
        custom_lang_broadband_model_id,
        '{Grammar-name}',
        grammar_file,
        'application/srgs'
    )
# Poll for grammar status.

### (Optional) print the language custom model

In [ ]:
speech_to_text.train_language_model(custom_lang_broadband_model_id)

### Check status of the custom language model -- has to be 
#### available

In [ ]:
# Get status of the language model - wait until it is 'available'
language_models = speech_to_text.list_language_models().get_result()
models = language_models["customizations"]
#print(json.dumps(language_models, indent=2))
for model in models:
    if model['customization_id'] == custom_lang_broadband_model_id:
        print (model['status'])

### (Optional) Print all the acoustic models

In [ ]:
acoustic_models = speech_to_text.list_acoustic_models().get_result()
print(json.dumps(acoustic_models, indent=2))

In [ ]:
custom_acoustic_broadband_model_id = '{Custom-Acoustic-Model-ID}'

### List all the audio files used for acoustic modeling

In [ ]:
audio_resources = speech_to_text.list_audio(custom_acoustic_broadband_model_id).get_result()
print(json.dumps(audio_resources, indent=2))

### (Optional) Add any additional audio file to the acoustic model

In [ ]:
from os.path import join, dirname

audioFilePath = '{Audio-File-Path}/filename.mp3'

with open(join(dirname('.'), './.', audioFilePath),
               'rb') as audio_file:
    speech_to_text.add_audio(
        custom_acoustic_broadband_model_id,
        '{acoustic-model-name}',
        audio_file,
        content_type='audio/mp3'
    )
# Poll for audio status.

### List the details of the acoustic model

In [ ]:
acoustic_model = speech_to_text.get_acoustic_model(custom_acoustic_broadband_model_id).get_result()
print(json.dumps(acoustic_model, indent=2))

### Upgrade the acoustic model

In [ ]:
speech_to_text.upgrade_acoustic_model(custom_acoustic_broadband_model_id)

### Train the acoustic model

In [ ]:
speech_to_text.train_acoustic_model(custom_acoustic_broadband_model_id)

###  Check status of the custom acoustic model -- has to be 
#### available

In [ ]:
# Get status of the acoustic model - wait until it is 'available'
acoustic_models = speech_to_text.list_acoustic_models().get_result()
models = acoustic_models["customizations"]

import time

statusNotAvailable = True
while statusNotAvailable:    
    for model in models:
        if model['customization_id'] == custom_acoustic_broadband_model_id: 
            if model['status'] == 'available':
                print(model['status'])
                statusNotAvailable = False
                break
            else:
                time.sleep(15)
                print(model['status'])

### Run transcription using the custom acoustic & language models

In [ ]:
files = ['{Location-Path}/filename.mp3']
for file in files:
    with open(join(dirname('.'), './.', file),
                   'rb') as audio_file:
        speech_recognition_results = speech_to_text.recognize(
            audio=audio_file,
            content_type='audio/mp3',
            model='en-US_BroadbandModel',
            customization_id=custom_lang_broadband_model_id,
            acoustic_customization_id=custom_acoustic_broadband_model_id,
            timestamps=True,
            word_alternatives_threshold=0.9,
            keywords=['mother coil', 'start length', 'drive side quarter', 'driver impression', 'severity'],
            keywords_threshold=0.5).get_result()
    print(json.dumps(speech_recognition_results, indent=2))

### Print the transcript from the custom model

In [ ]:
transcript2 = ''
for chunks in speech_recognition_results['results']:
    if 'alternatives' in chunks.keys():
        alternatives = chunks['alternatives'][0]
        if 'transcript' in alternatives:
            transcript2 = transcript2 + alternatives['transcript']
print(transcript2)

In [ ]:
files = ['{Location-Path}/filename.mp3']
for file in files:
    with open(join(dirname('.'), './.', file),
                   'rb') as audio_file:
        speech_recognition_results = speech_to_text.recognize(
            audio=audio_file,
            content_type='audio/mp3',
            model='en-US_BroadbandModel',
            customization_id=custom_lang_broadband_model_id,
            acoustic_customization_id=custom_acoustic_broadband_model_id,
            timestamps=True,
            word_alternatives_threshold=0.9,
            keywords=['mother coil', 'start length', 'drive side quarter', 'driver impression', 'severity'],
            keywords_threshold=0.5).get_result()
    print(json.dumps(speech_recognition_results, indent=2))

In [ ]:
transcript3 = ''
for chunks in speech_recognition_results['results']:
    if 'alternatives' in chunks.keys():
        alternatives = chunks['alternatives'][0]
        if 'transcript' in alternatives:
            transcript3 = transcript3 + alternatives['transcript']
print(transcript3)